In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style preferences
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Load the dataset
df = pd.read_csv('../data/processed/fully_cleaned_us_regional_sales_data.csv')

# Data preprocessing
df['OrderDate'] = pd.to_datetime(df['OrderDate'], errors='coerce')
df['Order Quarter'] = df['OrderDate'].dt.to_period('Q')
df['Order Month'] = df['OrderDate'].dt.month
df['Order Year'] = df['OrderDate'].dt.year

# Calculate delivery metrics
df['Processing_Time'] = (df['Ship Day'] - df['Order Day']).clip(lower=0)
df['Delivery_Time'] = (df['Delivery Day'] - df['Ship Day']).clip(lower=0)
df['Total_Fulfillment_Time'] = (df['Delivery Day'] - df['Order Day']).clip(lower=0)


In [ ]:
print("="*80)
print("STRATEGIC BUSINESS INTELLIGENCE VISUALIZATIONS")
print("="*80)

# Create output directory for saving plots
import os
output_dir = '../reports/figures/visualizations'
os.makedirs(output_dir, exist_ok=True)

''' SECTION 1: EXECUTIVE DASHBOARD VISUALIZATIONS '''
print("\n1. CREATING EXECUTIVE DASHBOARD VISUALIZATIONS")
print("-" * 60)

# 1.1 Channel Performance Overview Dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Revenue by Channel', 'Profit Margin by Channel', 
                   'Order Volume by Channel', 'Market Share Distribution'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'pie'}]]
)

# Revenue by Channel
channel_revenue = df.groupby('Sales Channel')['Total Revenue'].sum().sort_values(ascending=False)
fig.add_trace(go.Bar(x=channel_revenue.index, y=channel_revenue.values, 
                     name='Revenue', marker_color='lightblue'), row=1, col=1)

# Profit Margin by Channel
channel_margin = df.groupby('Sales Channel')['Profit Margin'].mean().sort_values(ascending=False)
fig.add_trace(go.Bar(x=channel_margin.index, y=channel_margin.values, 
                     name='Profit Margin', marker_color='lightgreen'), row=1, col=2)

# Order Volume by Channel
channel_orders = df.groupby('Sales Channel')['OrderNumber'].nunique().sort_values(ascending=False)
fig.add_trace(go.Bar(x=channel_orders.index, y=channel_orders.values, 
                     name='Orders', marker_color='lightsalmon'), row=2, col=1)

# Market Share Pie
market_share = (channel_revenue / channel_revenue.sum() * 100).round(1)
fig.add_trace(go.Pie(labels=market_share.index, values=market_share.values, 
                     name="Market Share"), row=2, col=2)

fig.update_layout(height=800, title_text="Executive Dashboard - Channel Performance Overview", 
                  showlegend=False)
fig.write_html(f'{output_dir}/executive_dashboard.html')
fig.show()

# 1.2 Financial Performance KPI Dashboard
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Total Revenue Trend
monthly_revenue = df.groupby(['Order Year', 'Order Month'])['Total Revenue'].sum().reset_index()
monthly_revenue['Date'] = pd.to_datetime(
    monthly_revenue[['Order Year', 'Order Month']].rename(
        columns={'Order Year': 'year', 'Order Month': 'month'}
    ).assign(day=1)
)
ax1.plot(monthly_revenue['Date'], monthly_revenue['Total Revenue'], marker='o', linewidth=2)
ax1.set_title('Monthly Revenue Trend', fontsize=14, fontweight='bold')
ax1.set_ylabel('Revenue ($)')
ax1.tick_params(axis='x', rotation=45)

# Profitability by Channel
channel_profit = df.groupby('Sales Channel')['Total Profit'].sum().sort_values(ascending=True)
ax2.barh(channel_profit.index, channel_profit.values, color='green', alpha=0.7)
ax2.set_title('Total Profit by Channel', fontsize=14, fontweight='bold')
ax2.set_xlabel('Profit ($)')

# Revenue vs Profit Scatter
channel_metrics = df.groupby('Sales Channel').agg({
    'Total Revenue': 'sum',
    'Total Profit': 'sum'
}).reset_index()
scatter = ax3.scatter(channel_metrics['Total Revenue'], channel_metrics['Total Profit'], 
                     s=100, alpha=0.6, c=range(len(channel_metrics)), cmap='viridis')
ax3.set_title('Revenue vs Profit by Channel', fontsize=14, fontweight='bold')
ax3.set_xlabel('Total Revenue ($)')
ax3.set_ylabel('Total Profit ($)')
for i, channel in enumerate(channel_metrics['Sales Channel']):
    ax3.annotate(channel, (channel_metrics.iloc[i]['Total Revenue'], 
                          channel_metrics.iloc[i]['Total Profit']), fontsize=9)

# Growth Rate Analysis
if len(df['Order Year'].unique()) > 1:
    yearly_growth = df.groupby(['Order Year', 'Sales Channel'])['Total Revenue'].sum().reset_index()
    yearly_pivot = yearly_growth.pivot(index='Sales Channel', columns='Order Year', values='Total Revenue').fillna(0)
    if yearly_pivot.shape[1] > 1:
        years = sorted(yearly_pivot.columns)
        growth_rate = ((yearly_pivot[years[-1]] - yearly_pivot[years[-2]]) / yearly_pivot[years[-2]] * 100).fillna(0)
        growth_rate.plot(kind='bar', ax=ax4, color='orange', alpha=0.7)
        ax4.set_title('YoY Growth Rate by Channel (%)', fontsize=14, fontweight='bold')
        ax4.set_ylabel('Growth Rate (%)')
        ax4.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(f'{output_dir}/financial_kpi_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
''' SECTION 2: GROWTH & TREND ANALYSIS VISUALIZATIONS '''
print("\n2. CREATING GROWTH & TREND VISUALIZATIONS")
print("-" * 60)

# 2.1 Time-Series Growth Analysis
fig = make_subplots(
    rows=2, cols=1,
    subplot_titles=('Monthly Revenue Trend by Channel', 'Cumulative Revenue Growth'),
    vertical_spacing=0.1
)

# Monthly trends by channel
monthly_channel = df.groupby(['Order Month', 'Sales Channel'])['Total Revenue'].sum().reset_index()
for channel in df['Sales Channel'].unique():
    channel_data = monthly_channel[monthly_channel['Sales Channel'] == channel]
    fig.add_trace(go.Scatter(x=channel_data['Order Month'], y=channel_data['Total Revenue'],
                            mode='lines+markers', name=channel, line=dict(width=3)), row=1, col=1)

# Cumulative revenue
df_sorted = df.sort_values('OrderDate')
for channel in df['Sales Channel'].unique():
    channel_df = df_sorted[df_sorted['Sales Channel'] == channel]
    channel_df['Cumulative_Revenue'] = channel_df['Total Revenue'].cumsum()
    fig.add_trace(go.Scatter(x=channel_df['OrderDate'], y=channel_df['Cumulative_Revenue'],
                            mode='lines', name=f'{channel} (Cumulative)', 
                            line=dict(dash='dot')), row=2, col=1)

fig.update_layout(height=800, title_text="Growth & Trend Analysis")
fig.write_html(f'{output_dir}/growth_trend_analysis.html')
fig.show()

# 2.2 Seasonal Analysis Heatmap
plt.figure(figsize=(14, 8))
monthly_channel_pivot = monthly_channel.pivot(index='Sales Channel', columns='Order Month', values='Total Revenue')
sns.heatmap(monthly_channel_pivot, annot=True, fmt='.0f', cmap='YlOrRd', 
            cbar_kws={'label': 'Revenue ($)'})
plt.title('Seasonal Revenue Heatmap by Channel', fontsize=16, fontweight='bold')
plt.xlabel('Month')
plt.ylabel('Sales Channel')
plt.tight_layout()
plt.savefig(f'{output_dir}/seasonal_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
''' SECTION 3: DELIVERY PERFORMANCE VISUALIZATIONS '''
print("\n3. CREATING DELIVERY PERFORMANCE VISUALIZATIONS")
print("-" * 60)

# 3.1 Delivery Performance Dashboard
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Average delivery times by channel
delivery_avg = df.groupby('Sales Channel')[['Processing_Time', 'Delivery_Time', 'Total_Fulfillment_Time']].mean()
delivery_avg.plot(kind='bar', ax=ax1, width=0.8)
ax1.set_title('Average Delivery Times by Channel', fontsize=14, fontweight='bold')
ax1.set_ylabel('Days')
ax1.tick_params(axis='x', rotation=45)
ax1.legend(['Processing', 'Delivery', 'Total Fulfillment'])

# Delivery time distribution
ax2.hist([df['Total_Fulfillment_Time']], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
ax2.axvline(df['Total_Fulfillment_Time'].mean(), color='red', linestyle='--', 
            label=f'Mean: {df["Total_Fulfillment_Time"].mean():.1f} days')
ax2.set_title('Distribution of Total Fulfillment Time', fontsize=14, fontweight='bold')
ax2.set_xlabel('Days')
ax2.set_ylabel('Frequency')
ax2.legend()

# Service level achievement
service_levels = df.groupby('Sales Channel').agg({
    'Total_Fulfillment_Time': [
        lambda x: (x <= 3).mean() * 100,
        lambda x: (x <= 7).mean() * 100,
        lambda x: (x <= 14).mean() * 100
    ]
}).round(1)
service_levels.columns = ['≤3 days', '≤7 days', '≤14 days']
service_levels.plot(kind='bar', ax=ax3, width=0.8)
ax3.set_title('Service Level Achievement (%)', fontsize=14, fontweight='bold')
ax3.set_ylabel('Percentage (%)')
ax3.tick_params(axis='x', rotation=45)

# Warehouse efficiency
warehouse_efficiency = df.groupby('WarehouseCode')['Total_Fulfillment_Time'].mean().sort_values()
ax4.barh(warehouse_efficiency.index, warehouse_efficiency.values, color='lightcoral')
ax4.set_title('Average Fulfillment Time by Warehouse', fontsize=14, fontweight='bold')
ax4.set_xlabel('Days')

plt.tight_layout()
plt.savefig(f'{output_dir}/delivery_performance_dashboard.png', dpi=300, bbox_inches='tight')
plt.show()

# 3.2 Delivery Performance Scatter Matrix
delivery_metrics = df.groupby('Sales Channel').agg({
    'Processing_Time': 'mean',
    'Delivery_Time': 'mean',
    'Total_Fulfillment_Time': 'mean',
    'Total Revenue': 'sum'
}).reset_index()

fig = px.scatter_matrix(delivery_metrics, 
                       dimensions=['Processing_Time', 'Delivery_Time', 'Total_Fulfillment_Time'],
                       color='Sales Channel',
                       title="Delivery Performance Correlation Matrix")
fig.write_html(f'{output_dir}/delivery_correlation_matrix.html')
fig.show()

In [ ]:
''' SECTION 4: PRODUCT INTELLIGENCE VISUALIZATIONS '''
print("\n4. CREATING PRODUCT INTELLIGENCE VISUALIZATIONS")
print("-" * 60)

# 4.1 Product Portfolio Analysis (BCG Matrix Style)
product_metrics = df.groupby('_ProductID').agg({
    'Total Revenue': 'sum',
    'OrderNumber': 'count',
    'Profit Margin': 'mean',
    'Order Quantity': 'sum'
})

product_metrics['Market_Share'] = product_metrics['Total Revenue'] / product_metrics['Total Revenue'].sum() * 100
product_metrics['Order_Frequency'] = product_metrics['OrderNumber']

# BCG Matrix visualization
plt.figure(figsize=(12, 8))
scatter = plt.scatter(product_metrics['Market_Share'], product_metrics['Order_Frequency'], 
                     s=product_metrics['Total Revenue']/10000, alpha=0.6, 
                     c=product_metrics['Profit Margin'], cmap='RdYlGn')

plt.axhline(y=product_metrics['Order_Frequency'].median(), color='gray', linestyle='--', alpha=0.7)
plt.axvline(x=product_metrics['Market_Share'].median(), color='gray', linestyle='--', alpha=0.7)

# Add quadrant labels
plt.text(product_metrics['Market_Share'].max()*0.8, product_metrics['Order_Frequency'].max()*0.9, 
         'STARS', fontsize=12, fontweight='bold', ha='center')
plt.text(product_metrics['Market_Share'].max()*0.8, product_metrics['Order_Frequency'].min()*1.1, 
         'CASH COWS', fontsize=12, fontweight='bold', ha='center')
plt.text(product_metrics['Market_Share'].min()*1.2, product_metrics['Order_Frequency'].max()*0.9, 
         'QUESTION MARKS', fontsize=12, fontweight='bold', ha='center')
plt.text(product_metrics['Market_Share'].min()*1.2, product_metrics['Order_Frequency'].min()*1.1, 
         'DOGS', fontsize=12, fontweight='bold', ha='center')

plt.xlabel('Market Share (%)', fontsize=12)
plt.ylabel('Order Frequency', fontsize=12)
plt.title('Product Portfolio Analysis (BCG Matrix)', fontsize=16, fontweight='bold')
plt.colorbar(scatter, label='Profit Margin (%)')
plt.tight_layout()
plt.savefig(f'{output_dir}/product_portfolio_bcg.png', dpi=300, bbox_inches='tight')
plt.show()

# 4.2 Top Products Revenue Waterfall Chart
top_products = df.groupby('_ProductID')['Total Revenue'].sum().sort_values(ascending=False).head(10)

fig = go.Figure(go.Waterfall(
    name="Product Revenue",
    orientation="v",
    measure=["relative"] * len(top_products),
    x=[f"Product {pid}" for pid in top_products.index],
    text=[f"${val:,.0f}" for val in top_products.values],
    y=top_products.values,
    connector={"line":{"color":"rgb(63, 63, 63)"}},
))

fig.update_layout(
    title="Top 10 Products Revenue Contribution",
    showlegend=False,
    height=600
)
fig.write_html(f'{output_dir}/top_products_waterfall.html')
fig.show()

In [ ]:
''' SECTION 5: CHANNEL PERFORMANCE ANALYSIS '''
print("\n5. CREATING CHANNEL PERFORMANCE VISUALIZATIONS")
print("-" * 60)

# 5.1 Channel Performance Radar Chart
channel_performance = df.groupby('Sales Channel').agg({
    'Total Revenue': 'sum',
    'Total Profit': 'sum',
    'Profit Margin': 'mean',
    'Order Quantity': 'sum',
    'Total_Fulfillment_Time': 'mean'
}).round(2)

# Normalize metrics for radar chart (0-100 scale)
normalized_performance = channel_performance.copy()
for col in normalized_performance.columns:
    if col == 'Total_Fulfillment_Time':  # Lower is better for delivery time
        normalized_performance[col] = 100 - (normalized_performance[col] - normalized_performance[col].min()) / (normalized_performance[col].max() - normalized_performance[col].min()) * 100
    else:  # Higher is better for other metrics
        normalized_performance[col] = (normalized_performance[col] - normalized_performance[col].min()) / (normalized_performance[col].max() - normalized_performance[col].min()) * 100

# Create radar chart
fig = go.Figure()

for channel in normalized_performance.index:
    fig.add_trace(go.Scatterpolar(
        r=normalized_performance.loc[channel].values.tolist() + [normalized_performance.loc[channel].values[0]],
        theta=list(normalized_performance.columns) + [normalized_performance.columns[0]],
        fill='toself',
        name=channel,
        line=dict(width=2)
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 100]
        )),
    showlegend=True,
    title="Channel Performance Radar Chart (Normalized Scores)",
    height=600
)
fig.write_html(f'{output_dir}/channel_performance_radar.html')
fig.show()

# 5.2 Channel Efficiency Analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Revenue per Order
channel_efficiency = df.groupby('Sales Channel').agg({
    'Total Revenue': 'sum',
    'OrderNumber': 'nunique'
})
channel_efficiency['Revenue_per_Order'] = channel_efficiency['Total Revenue'] / channel_efficiency['OrderNumber']
channel_efficiency['Revenue_per_Order'].sort_values(ascending=True).plot(kind='barh', ax=ax1, color='lightblue')
ax1.set_title('Revenue per Order by Channel', fontsize=14, fontweight='bold')
ax1.set_xlabel('Revenue per Order ($)')

# Profit Margin Distribution
df.boxplot(column='Profit Margin', by='Sales Channel', ax=ax2)
ax2.set_title('Profit Margin Distribution by Channel', fontsize=14, fontweight='bold')
ax2.set_xlabel('Sales Channel')
ax2.set_ylabel('Profit Margin (%)')
plt.suptitle('')  # Remove automatic title

plt.tight_layout()
plt.savefig(f'{output_dir}/channel_efficiency_analysis.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
''' SECTION 6: STRATEGIC INSIGHTS VISUALIZATIONS '''
print("\n6. CREATING STRATEGIC INSIGHTS VISUALIZATIONS")
print("-" * 60)

# 6.1 Risk-Return Analysis
fig, ax = plt.subplots(figsize=(12, 8))

# Calculate risk (volatility) and return for each channel
channel_risk_return = df.groupby(['Sales Channel', 'Order Month'])['Total Revenue'].sum().reset_index()
channel_stats = channel_risk_return.groupby('Sales Channel')['Total Revenue'].agg(['mean', 'std']).reset_index()
channel_stats.columns = ['Sales Channel', 'Average_Revenue', 'Revenue_Volatility']

# Create bubble chart
channel_profit = df.groupby('Sales Channel')['Total Profit'].sum().reset_index()
merged_stats = channel_stats.merge(channel_profit, on='Sales Channel')

scatter = ax.scatter(merged_stats['Revenue_Volatility'], merged_stats['Average_Revenue'],
                    s=merged_stats['Total Profit']/50000, alpha=0.6, 
                    c=range(len(merged_stats)), cmap='viridis')

# Add channel labels
for i, channel in enumerate(merged_stats['Sales Channel']):
    ax.annotate(channel, (merged_stats.iloc[i]['Revenue_Volatility'], 
                         merged_stats.iloc[i]['Average_Revenue']), 
                fontsize=10, ha='center')

ax.set_xlabel('Revenue Volatility (Risk)', fontsize=12)
ax.set_ylabel('Average Monthly Revenue (Return)', fontsize=12)
ax.set_title('Channel Risk-Return Analysis\n(Bubble size = Total Profit)', fontsize=14, fontweight='bold')

# Add quadrant lines
ax.axhline(y=merged_stats['Average_Revenue'].median(), color='gray', linestyle='--', alpha=0.5)
ax.axvline(x=merged_stats['Revenue_Volatility'].median(), color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.savefig(f'{output_dir}/risk_return_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

# 6.2 Performance Summary Dashboard
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=('Channel ROI Ranking', 'Delivery Performance Score', 'Product Diversity',
                   'Growth Trajectory', 'Market Share Trend', 'Profitability Index'),
    specs=[[{'type': 'bar'}, {'type': 'bar'}, {'type': 'bar'}],
           [{'type': 'scatter'}, {'type': 'scatter'}, {'type': 'bar'}]]
)

# Channel ROI
channel_roi = df.groupby('Sales Channel').agg({'Total Profit': 'sum', 'Total Cost': 'sum'})
channel_roi['ROI'] = (channel_roi['Total Profit'] / channel_roi['Total Cost'] * 100).round(2)
roi_sorted = channel_roi['ROI'].sort_values(ascending=False)
fig.add_trace(go.Bar(x=roi_sorted.index, y=roi_sorted.values, name='ROI'), row=1, col=1)

# Delivery Performance Score (composite)
delivery_score = df.groupby('Sales Channel')['Total_Fulfillment_Time'].mean()
delivery_score = ((delivery_score.max() - delivery_score) / delivery_score.max() * 100).round(2)
fig.add_trace(go.Bar(x=delivery_score.index, y=delivery_score.values, name='Delivery Score'), row=1, col=2)

# Product Diversity
product_diversity = df.groupby('Sales Channel')['_ProductID'].nunique().sort_values(ascending=False)
fig.add_trace(go.Bar(x=product_diversity.index, y=product_diversity.values, name='Products'), row=1, col=3)

# Add more subplots with relevant metrics...
if len(df['Order Year'].unique()) > 1:
    # Growth trajectory
    yearly_channel = df.groupby(['Order Year', 'Sales Channel'])['Total Revenue'].sum().reset_index()
    yearly_channel['Order Year'] = yearly_channel['Order Year'].astype(int)
    for channel in df['Sales Channel'].unique()[:3]:  # Top 3 channels
        channel_data = yearly_channel[yearly_channel['Sales Channel'] == channel]
        fig.add_trace(go.Scatter(x=channel_data['Order Year'], y=channel_data['Total Revenue'],
                               mode='lines+markers', name=channel), row=2, col=1)

# Market Share Trend (row=2, col=2)
market_share_trend = df.groupby(['Order Year', 'Sales Channel'])['Total Revenue'].sum().reset_index()
market_share_trend['Order Year'] = market_share_trend['Order Year'].astype(int)
total_yearly_revenue = market_share_trend.groupby('Order Year')['Total Revenue'].transform('sum')
market_share_trend['Market_Share'] = (market_share_trend['Total Revenue'] / total_yearly_revenue * 100).round(2)
for channel in df['Sales Channel'].unique():
    channel_data = market_share_trend[market_share_trend['Sales Channel'] == channel]
    fig.add_trace(go.Scatter(x=channel_data['Order Year'], y=channel_data['Market_Share'],
                             mode='lines+markers', name=f"{channel} Market Share"), row=2, col=2)

# Profitability Index (row=2, col=3)
profitability_index = df.groupby('Sales Channel')['Profit Margin'].mean().sort_values(ascending=False)
fig.add_trace(go.Bar(x=profitability_index.index, y=profitability_index.values, name='Profitability Index'), row=2, col=3)

fig.update_layout(height=1000, title_text="Strategic Performance Summary Dashboard", 
                  showlegend=False)
fig.write_html(f'{output_dir}/strategic_summary_dashboard.html')
fig.show()


In [ ]:
''' SECTION 7: EXECUTIVE SUMMARY REPORT VISUALS '''
print("\n7. CREATING EXECUTIVE SUMMARY VISUALS")
print("-" * 60)

# 7.1 One-Page Executive Summary
fig = plt.figure(figsize=(20, 24))

# Create a complex subplot layout for executive summary
gs = fig.add_gridspec(6, 4, height_ratios=[1, 1, 1, 1, 1, 0.5], hspace=0.3, wspace=0.3)

# Top KPIs
ax_kpi = fig.add_subplot(gs[0, :])
total_revenue = df['Total Revenue'].sum()
total_orders = df['OrderNumber'].nunique()
avg_order_value = df['Total Revenue'].mean()
total_profit = df['Total Profit'].sum()

kpi_data = [total_revenue, total_orders, avg_order_value, total_profit]
kpi_labels = ['Total Revenue\n${:,.0f}'.format(total_revenue), 
              'Total Orders\n{:,}'.format(total_orders),
              'Avg Order Value\n${:.2f}'.format(avg_order_value),
              'Total Profit\n${:,.0f}'.format(total_profit)]

bars = ax_kpi.bar(range(len(kpi_data)), [1]*len(kpi_data), color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
ax_kpi.set_ylim(0, 1.2)
ax_kpi.set_xlim(-0.5, len(kpi_data)-0.5)
ax_kpi.set_xticks(range(len(kpi_data)))
ax_kpi.set_xticklabels(kpi_labels, fontsize=12, fontweight='bold')
ax_kpi.set_title('KEY PERFORMANCE INDICATORS', fontsize=16, fontweight='bold', pad=20)
ax_kpi.set_yticks([])

# Channel Performance (2nd row)
ax1 = fig.add_subplot(gs[1, :2])
channel_revenue.plot(kind='bar', ax=ax1, color='skyblue')
ax1.set_title('Revenue by Channel', fontsize=14, fontweight='bold')
ax1.tick_params(axis='x', rotation=45)

ax2 = fig.add_subplot(gs[1, 2:])
channel_margin.plot(kind='bar', ax=ax2, color='lightgreen')
ax2.set_title('Profit Margin by Channel', fontsize=14, fontweight='bold')
ax2.tick_params(axis='x', rotation=45)

# Growth and Trends (3rd row)
ax3 = fig.add_subplot(gs[2, :2])
monthly_revenue.plot(x='Date', y='Total Revenue', ax=ax3, marker='o')
ax3.set_title('Monthly Revenue Trend', fontsize=14, fontweight='bold')

ax4 = fig.add_subplot(gs[2, 2:])
delivery_avg['Total_Fulfillment_Time'].plot(kind='bar', ax=ax4, color='orange')
ax4.set_title('Avg Fulfillment Time by Channel', fontsize=14, fontweight='bold')
ax4.tick_params(axis='x', rotation=45)

# Product and Risk Analysis (4th row)
ax5 = fig.add_subplot(gs[3, :2])
top_products.head(5).plot(kind='bar', ax=ax5, color='purple')
ax5.set_title('Top 5 Products by Revenue', fontsize=14, fontweight='bold')

ax6 = fig.add_subplot(gs[3, 2:])
ax6.scatter(merged_stats['Revenue_Volatility'], merged_stats['Average_Revenue'], 
           s=100, alpha=0.7, c='red')
ax6.set_title('Risk-Return Analysis', fontsize=14, fontweight='bold')
ax6.set_xlabel('Risk (Volatility)')
ax6.set_ylabel('Return (Avg Revenue)')

# Strategic Recommendations (5th row)
ax7 = fig.add_subplot(gs[4, :])
recommendations_text = """
STRATEGIC RECOMMENDATIONS:

1. CHANNEL OPTIMIZATION: Focus investment on highest ROI channels while improving delivery times for competitive advantage

2. PRODUCT PORTFOLIO: Prioritize Star products for growth, maintain Cash Cows, and evaluate underperforming products

3. OPERATIONAL EFFICIENCY: Optimize warehouse-channel allocation and implement performance monitoring systems

4. RISK MANAGEMENT: Diversify revenue streams and monitor volatile channels for stability improvements

5. GROWTH ACCELERATION: Leverage seasonal patterns and expand successful channel strategies
"""

ax7.text(0.05, 0.95, recommendations_text, transform=ax7.transAxes, fontsize=11, 
         verticalalignment='top', bbox=dict(boxstyle="round,pad=0.5", facecolor='lightgray', alpha=0.8))
ax7.set_xlim(0, 1)
ax7.set_ylim(0, 1)
ax7.axis('off')
ax7.set_title('STRATEGIC RECOMMENDATIONS', fontsize=16, fontweight='bold')

# Footer
ax_footer = fig.add_subplot(gs[5, :])
ax_footer.text(0.5, 0.5, f'Business Intelligence Report | Generated: {datetime.now().strftime("%Y-%m-%d %H:%M")} | Data Period: {df["OrderDate"].min().strftime("%Y-%m-%d")} to {df["OrderDate"].max().strftime("%Y-%m-%d")}', 
               transform=ax_footer.transAxes, ha='center', va='center', fontsize=10, style='italic')
ax_footer.axis('off')

plt.suptitle('EXECUTIVE BUSINESS INTELLIGENCE SUMMARY', fontsize=20, fontweight='bold', y=0.98)
plt.savefig(f'{output_dir}/executive_summary_report.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

print(f"\n✅ All visualizations have been saved to: {output_dir}/")
print("\nFiles created:")
print("- executive_dashboard.html")
print("- financial_kpi_dashboard.png")
print("- growth_trend_analysis.html")
print("- seasonal_heatmap.png")
print("- delivery_performance_dashboard.png")
print("- delivery_correlation_matrix.html")
print("- product_portfolio_bcg.png")
print("- top_products_waterfall.html")
print("- channel_performance_radar.html")
print("- channel_efficiency_analysis.png")
print("- risk_return_analysis.png")
print("- strategic_summary_dashboard.html")
print("- executive_summary_report.png")

print("\n" + "="*80)
print("STRATEGIC VISUALIZATION SUITE COMPLETE")
print("="*80)